In [1]:
import re
import os
import json
import nltk
import scipy
import numpy as np
import pandas as pd

from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from itertools import chain
from tqdm import tqdm

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
diag = pd.read_csv('diag.csv', sep=';')
faq = pd.read_csv('faq.csv', sep=';')

/home/ubuntu/miniconda3/envs/p35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
diag = diag.fillna('')
faq = faq.fillna('')

In [5]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.9)

In [6]:
tfidf.fit(list(diag.token_question) + list(diag.token_answer) + list(faq.question) + list(faq.answer))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
diag[diag.question.str.find('?')!=-1].token_answer[:2]

0    здравствовать только полностью закрывать вклад
1                                                  
Name: token_answer, dtype: object

In [10]:
# dists = cosine_similarity(tfidf.transform(diag.token_answer), tfidf.transform(faq.answer))

In [11]:
def text_dist(text, text_list):
    sim = cosine_similarity(tfidf.transform(text), tfidf.transform(text_list))[0]
    return [text[0], text_list[sim==max(sim)].values[0], max(sim)]

In [16]:
sims = []
x = diag[diag.question.str.find('?')!=-1].token_answer.values
for i in tqdm(range(len(x))):
    sims.append(text_dist([x[i]], faq.answer))

100%|██████████| 57489/57489 [07:15<00:00, 132.10it/s]


In [28]:
sim_data = pd.DataFrame(sims, columns = ['token_answer', 'answer', 'distance'])

In [39]:
# sim_data[(sim_data.distance < 0.1)&(sim_data.distance > 0.05)]
sim_data[(sim_data.distance > 0.7)&(~sim_data.answer.isin(['да конечно', 'да быть', 'да']))].merge(
    diag[diag.question.str.find('?')!=-1][['question','token_answer']], on='token_answer').merge(faq, on='answer').drop_duplicates(
subset = ['answer', 'distance', 'question_x']).to_csv('add_questions.csv',index=False)
    

In [48]:
sims = []
x = diag[diag.question.str.find('?')!=-1].token_question.values
for i in tqdm(range(len(x))):
    sims.append(text_dist([x[i]], faq.answer))

100%|██████████| 57489/57489 [07:16<00:00, 131.57it/s]


In [49]:
sim_data = pd.DataFrame(sims, columns = ['token_question', 'answer', 'distance'])

In [52]:
sim_data[(sim_data.distance > 0.75)]

,token_question,answer,distance
5646,да,да быть,0.810542
6040,да быть,да быть,1.000000
10729,да,да быть,0.810542
12062,да,да быть,0.810542
12063,что да да быть комиссия,да быть,0.777776
17505,да,да быть,0.810542
21016,да,да быть,0.810542
26017,да,да быть,0.810542
29549,да,да быть,0.810542
35906,да,да быть,0.810542


In [50]:
sim_data[(sim_data.distance > 0.75)].merge(
diag[diag.question.str.find('?')!=-1][['question','token_question']], on='token_question').merge(faq, on='question')

,token_question,answer_x,distance,question,answer_y


In [51]:
# sim_data[(sim_data.distance > 0.7)].merge(
#     diag[diag.question.str.find('?')!=-1][['question','token_question']], on='token_question').merge(faq, on='question').drop_duplicates(
# subset = ['answer', 'distance', 'question_x']).to_csv('add_questions2.csv',index=False)
    

In [ ]:
sims = []
x = diag[diag.question.str.find('?')!=-1].token_answer.values
for i in tqdm(range(len(x))):
    sims.append(text_dist([x[i]], faq.answer))